In [1]:


%matplotlib inline

##THIS IS FOR ACCESSING THE CONCORDE TSP SOLVER
from subprocess import Popen

import os
class tsp:
    def __init__(self, points=None):
        self.points = points if points else []
    def solve(self, filename='_tmp_'):
        with open('%s.dat' % filename, 'w') as fp:
            fp.write('TYPE : TSP\nDIMENSION : %d\n' % len(self.points))
            fp.write('EDGE_WEIGHT_TYPE : GEO\nNODE_COORD_SECTION\n')
            for i, (x, y) in enumerate(self.points):
                fp.write('%d %g %g\n' % (i, x, y))
            fp.write('EOF\n')
        prc = Popen('./concorde   -m %s.dat' % filename, shell=True)
        prc.wait()
        with open('%s.sol' % filename) as fp:
            ll = (''.join(fp)).split()
        self.result = [int(s) for s in ll[1:]]
        for s in ('%s.dat', '%s.sol'): os.remove(s % filename)

            
            

In [2]:
##THIS I FOR ACCESSING THE CONCORDE TSP SOLVER BZCKUP,SWITCHING TO GEO
#from subprocess import Popen
#import os
#class tsp:
#    def __init__(self, points=None):
#        self.points = points if points else []
#    def solve(self, filename='_tmp_'):
#        with open('%s.dat' % filename, 'w') as fp:
#            fp.write('TYPE : TSP\nDIMENSION : %d\n' % len(self.points))
#            fp.write('EDGE_WEIGHT_TYPE : EUC_2D\nNODE_COORD_SECTION\n')
#            for i, (x, y) in enumerate(self.points):
#                fp.write('%d %g %g\n' % (i, x, y))
#            fp.write('EOF\n')
#        prc = Popen('./concorde -m  %s.dat' % filename, shell=True)
#        prc.wait()
#        with open('%s.sol' % filename) as fp:
#            ll = (''.join(fp)).split()
#        self.result = [int(s) for s in ll[1:]]
#        for s in ('%s.dat', '%s.sol'): os.remove(s % filename)

In [3]:
import geopy
import pickle
import json
import pymongo
from geopy.distance import vincenty
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from slugify import slugify
import math
from numbers import Number

# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["bandsintown"]

# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

import numpy as np
from matplotlib import pyplot as plt


In [4]:
final_embeddings=[]
totalDists=0
totalOptDists=0
totalOptDistsMin=0
#for artist in db.minedArtists.find({ 'name' : 'David Guetta' }) :
#for artist in db.minedArtists.find({ 'name' : 'Vokab Kompany' }) :
#for artist in db.minedArtists.find({ 'name' : 'Ural Kosaken Chor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Leo Kottke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Björk' }) :
#for artist in db.minedArtists.find({ 'name' : 'Deerhoof' }) :
#for artist in db.minedArtists.find({ 'name' : 'Legends' }) :
#for artist in db.minedArtists.find({ 'name' : 'Laidback Luke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Andy Moor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Mia' }) :
#for artist in db.minedArtists.find({ 'name' : 'Dead Kennedys' }) :
#for artist in db.minedArtists.find({ 'name' : 'House of Pain' }) :
for artist in db.minedArtists.find(no_cursor_timeout=True) :

 if os.path.exists("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        print "SKIPPED",artist["name"]
        #print slugify(artist["name"])
        with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          temp= pickle.load( handle)
        #print temp
        tmppp =0
        
        totalArtistTourDist=0
        totalArtistOptTourDist=0
        totalArtistTourDist=temp["totalArtistTourDist"]
        totalArtistOptTourDist=temp["totalArtistOptTourDist"]
        
        totalDists+=totalArtistTourDist
        totalOptDists+=totalArtistOptTourDist
        if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
        else:
            tmppp=float(totalArtistOptTourDist)
        totalOptDistsMin+=tmppp
        #print totalArtistTourDist," / ",totalArtistOptTourDist
        #print totalDists," / ",totalOptDists," / ",totalOptDistsMin," / ",tmppp
        
        #pass
 else:
    

    results={}
    tourindice=0
    results["name"]=artist["name"]
    #print artist["name"]
    totalArtistTourDist=0
    totalArtistOptTourDist=0
    for tour in artist["tours"]:
        #print tour["gigs"][0]["datetime"].year
        #print tour["gigs"][-1]["datetime"].year
        #print int(math.ceil(len(tour["gigs"])/2))
        tourCoords= []
        tourdist=0
        tourdistOpt=0
        tourdistOpt2=0
        tourdistOpt3=0
        tourLength=0
        tourYear=0
        tourYearStart=0
        tourYearEnd=0
        tourYearMid=0
        t=0
        #print len(tour["tourCoords"])
        if len(tour["tourCoords"])>2:
            tourLength=int(math.ceil(len(tour["gigs"])))
            tourYearStart=tour["gigs"][0]["datetime"].year
            tourYearEnd=tour["gigs"][-1]["datetime"].year
            if tourYearStart == tourYearEnd :
                tourYear= tourYearStart
            else:
                tourYearMid= tourYearEnd=tour["gigs"][int(math.ceil(len(tour["gigs"])/2))]["datetime"].year
                if tourYearStart == tourYearMid :
                    tourYear= tourYearStart
                else:
                    tourYear= tourYearEnd
            
            tourindice+=1
            tourCoords=[tuple(tourCoord) for tourCoord in tour["tourCoords"]]
            #print tourCoords
            ###RERUN OPT,PICKING BETTER SOLUTION
            t=tsp(tourCoords)
            t.solve()
            #print "here"

            #print t.result
            #print t.points
            #print "long",len (t.result)
            i = 0
            while i < len (t.result)-1:
                res=t.result[i]
                res2=t.result[i+1]
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdistOpt= tourdistOpt + vincenty(t.points[res],t.points[res2]).kilometers
                
                
                i+=1
            
            j = 0
            while j < len (tourCoords)-1:
                
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdist= tourdist + vincenty(tourCoords[j],tourCoords[j+1]).kilometers
                j+=1
            
            #print "DEFAULT TOUR DIST",tourdist
            #print "TOURDISTOPTIMIZED",tourdistOpt
            totalArtistTourDist+=tourdist
            totalArtistOptTourDist+=tourdistOpt
            results[tourindice]={'tourdist':tourdist,'tourdistOpt':tourdistOpt,'tourLength' : tourLength , 'tourYear': tourYear}
    #print results
    tourInd=0
    resultsYears={}
    for tourInd in results:
        #print tourInd
        if tourInd != 'name':
            #print results[tourInd]['tourYear']
            if results[tourInd]['tourYear'] not in resultsYears:
                resultsYears[results[tourInd]['tourYear']]={'toursCount':0,'toursDist':0,'toursDistOpt':0,'toursLength':0}
            
            resultsYears[results[tourInd]['tourYear']]['toursCount']+=1
            resultsYears[results[tourInd]['tourYear']]['toursDist']=resultsYears[results[tourInd]['tourYear']]['toursDist']+results[tourInd]['tourdist']
            resultsYears[results[tourInd]['tourYear']]['toursDistOpt']=resultsYears[results[tourInd]['tourYear']]['toursDistOpt']+results[tourInd]['tourdistOpt']
            resultsYears[results[tourInd]['tourYear']]['toursLength']=resultsYears[results[tourInd]['tourYear']]['toursLength']+results[tourInd]['tourLength']
            
    #print resultsYears
    statNbYear=len(resultsYears)
    statMeanYearDist=0
    statMeanDistOpt=0
    statMeanYeartoursLength=0
    statMeanYeartoursCount=0
    #Calculating means
    for Year in resultsYears:
        statMeanYearDist+=resultsYears[Year]['toursDist']
        statMeanDistOpt+=resultsYears[Year]['toursDistOpt']
        statMeanYeartoursLength+=resultsYears[Year]['toursLength']
        statMeanYeartoursCount+=resultsYears[Year]['toursCount']
    #print statMeanYearDist
    #print statNbYear
    if statMeanYearDist > 0:
        statMeanYearDist=statMeanYearDist/statNbYear
        statMeanDistOpt=statMeanDistOpt/statNbYear
        statMeanYeartoursLength=statMeanYeartoursLength/statNbYear
        statMeanYeartoursCount=statMeanYeartoursCount/statNbYear
        resultsYears['statMeanYearDist']=statMeanYearDist
        resultsYears['statMeanYearDistOpt']=statMeanDistOpt
        resultsYears['statMeanYeartoursLength']=statMeanYeartoursLength
        resultsYears['statMeanYeartoursCount']=statMeanYeartoursCount
        resultsYears['statNbYear']=statNbYear
        
        for Year in resultsYears:
            if isinstance(Year, Number):
                #print Year
                resultsYears[Year]['stattoursDist']=float(resultsYears[Year]['toursDist']-resultsYears['statMeanYearDist'])/resultsYears['statMeanYearDist']
                resultsYears[Year]['stattoursDistOpt']=float(resultsYears[Year]['toursDistOpt']-resultsYears['statMeanYearDistOpt'])/resultsYears['statMeanYearDistOpt']
                resultsYears[Year]['stattoursLength']=float(resultsYears[Year]['toursLength']-resultsYears['statMeanYeartoursLength'])/resultsYears['statMeanYeartoursLength']
                resultsYears[Year]['stattoursCount']=float(resultsYears[Year]['toursCount']-resultsYears['statMeanYeartoursCount'])/resultsYears['statMeanYeartoursCount']
    
        
        
    #print "TOTAL DEFAULT ARTIST TOURS DISTS",totalArtistTourDist
    #print "TOTAL OPTIMIZED ARTIST TOURS DISTS",totalArtistOptTourDist 
    if totalArtistTourDist > 0 :
        OptArtistPercent=0
        OptArtistPercent=100 *(totalArtistTourDist-totalArtistOptTourDist) / totalArtistTourDist
        results["OptArtistPercent"]=OptArtistPercent
        #print "OPTIMIZED BY ", OptArtistPercent," %"
    #print"_______________________________"
    totalDists+=totalArtistTourDist
    totalOptDists+=totalArtistOptTourDist
    tmppp=0
    if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
    else:
            tmppp=float(totalArtistOptTourDist)
    totalOptDistsMin+=tmppp
    
    results["resultsYears"]=resultsYears
    results["totalArtistTourDist"]=totalArtistTourDist
    results["totalArtistOptTourDist"]=totalArtistOptTourDist
    
    with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #print results['resultsYears']
    #print totalDists," / ",totalArtistTourDist," / ",totalArtistOptTourDist," / ",totalOptDistsMin," / ",tmppp
print "***************************"
print "TOTAL ARTISTS TOURS DISTS",totalDists
print "TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS",totalOptDists 
print "TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS",totalOptDistsMin 
print "AUTO-OPTIMIZED BY ", 100 *(totalDists-totalOptDists) / totalDists," %"
print "MANUALLY OPTIMIZED BY ", 100 *(totalDists-totalOptDistsMin) / totalDists," %"
print "***************************"
with open("./RESULTSGLOBAL.obj", 'wb') as handle:
                            pickle.dump({"totalDists":totalDists,"totalOptDists":totalOptDists,"totalOptDistsMin":totalOptDistsMin,"autoOpt":(totalDists-totalOptDists) / totalDists,"manOpt":(totalDists-totalOptDistsMin) / totalDists}, handle, protocol=pickle.HIGHEST_PROTOCOL)

SKIPPED 조수미
SKIPPED Лили Иванова
SKIPPED ᵀᴴᴱ ᴼᴿᴵᴳᴵᴻᴬᴸ Funky Monkey
SKIPPED Калевала
SKIPPED Кристина Орбакайте
SKIPPED Дмитрий Дмитриевич Шостакович
SKIPPED Валерия
SKIPPED Евгений Игоревич Кисин
SKIPPED Би-2
SKIPPED Анна Юрьевна Нетребко
SKIPPED Μελίνα Ασλανίδου
SKIPPED ŚexPistols Eẋperience
SKIPPED ÜBERMENSCH (Rammstein Tribute Band)
SKIPPED Ørsted
SKIPPED Økapi
SKIPPED Özz Nûjen
SKIPPED ¡Mayday!
SKIPPED zahni
SKIPPED zac rome
SKIPPED yours are the only ears
SKIPPED youn caro
SKIPPED with Aaron Lee Tasjan
SKIPPED wilton
SKIPPED wellman
SKIPPED weRcalled
SKIPPED waldden
SKIPPED vinyltheatre
SKIPPED u z z v
SKIPPED twen
SKIPPED together PANGEA
SKIPPED thelastplaceyoulook
SKIPPED thee nosebleeds
SKIPPED thePhantom*
SKIPPED the one and only PPL MVR
SKIPPED the mexicats
SKIPPED the last five years
SKIPPED the hoyle brothers
SKIPPED the dove & the wolf
SKIPPED the Wrongs
SKIPPED the Van Burens
SKIPPED the Tantrums
SKIPPED the Small Glories
SKIPPED the Skyline
SKIPPED the Shrine
SKIPPED the

In [14]:
#final_embeddings=[]
totalDists=0
totalOptDists=0
totalOptDistsMin=0
#for artist in db.minedArtists.find({ 'name' : 'David Guetta' }) :
#for artist in db.minedArtists.find({ 'name' : 'Vokab Kompany' }) :
#for artist in db.minedArtists.find({ 'name' : 'Ural Kosaken Chor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Leo Kottke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Björk' }) :
#for artist in db.minedArtists.find({ 'name' : 'Deerhoof' }) :
#for artist in db.minedArtists.find({ 'name' : 'Legends' }) :
#for artist in db.minedArtists.find({ 'name' : 'Laidback Luke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Andy Moor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Mia' }) :
#for artist in db.minedArtists.find({ 'name' : 'Dead Kennedys' }) :
#for artist in db.minedArtists.find({ 'name' : 'House of Pain' }) :
for artist in db.minedArtists.find(no_cursor_timeout=True) :

 if os.path.exists("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        print "SKIPPED",artist["name"]
        print slugify(artist["name"])
        with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          temp= pickle.load( handle)
        #print temp
        tmppp =0
        
        totalArtistTourDist=0
        totalArtistOptTourDist=0
        totalArtistTourDist=temp["totalArtistTourDist"]
        totalArtistOptTourDist=temp["totalArtistOptTourDist"]
        
        totalDists+=totalArtistTourDist
        totalOptDists+=totalArtistOptTourDist
        if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
        else:
            tmppp=float(totalArtistOptTourDist)
        totalOptDistsMin+=tmppp
        #print totalArtistTourDist," / ",totalArtistOptTourDist
        print totalDists," / ",totalOptDists," / ",totalOptDistsMin," / ",tmppp
        
        #pass
 else:
    

    results={}
    tourindice=0
    results["name"]=artist["name"]
    print artist["name"]
    totalArtistTourDist=0
    totalArtistOptTourDist=0
    for tour in artist["tours"]:
        #print tour
        tourCoords= []
        tourdist=0
        tourdistOpt=0
        tourdistOpt2=0
        tourdistOpt3=0
        t=0
        #print len(tour["tourCoords"])
        if len(tour["tourCoords"])>2:
            tourindice+=1
            tourCoords=[tuple(tourCoord) for tourCoord in tour["tourCoords"]]
            #print tourCoords
            ###RERUN OPT,PICKING BETTER SOLUTION
            t=tsp(tourCoords)
            t.solve()
            #print "here"

            #print t.result
            #print t.points
            #print "long",len (t.result)
            i = 0
            while i < len (t.result)-1:
                res=t.result[i]
                res2=t.result[i+1]
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdistOpt= tourdistOpt + vincenty(t.points[res],t.points[res2]).kilometers
                
                
                i+=1
            
            j = 0
            while j < len (tourCoords)-1:
                
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdist= tourdist + vincenty(tourCoords[j],tourCoords[j+1]).kilometers
                j+=1
            
            #print "DEFAULT TOUR DIST",tourdist
            #print "TOURDISTOPTIMIZED",tourdistOpt
            totalArtistTourDist+=tourdist
            totalArtistOptTourDist+=tourdistOpt
            results[tourindice]={'tourdist':tourdist,'tourdistOpt':tourdistOpt}
    #print results
    
    print "TOTAL DEFAULT ARTIST TOURS DISTS",totalArtistTourDist
    print "TOTAL OPTIMIZED ARTIST TOURS DISTS",totalArtistOptTourDist 
    if totalArtistTourDist > 0 :
        OptArtistpercent=0
        
        OptArtistpercent=100 *(totalArtistTourDist-totalArtistOptTourDist) / totalArtistTourDist
        print "OPTIMIZED BY ", ," %"
    print"_______________________________"
    totalDists+=totalArtistTourDist
    totalOptDists+=totalArtistOptTourDist
    tmppp=0
    if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
    else:
            tmppp=float(totalArtistOptTourDist)
    totalOptDistsMin+=tmppp
    

    results["totalArtistTourDist"]=totalArtistTourDist
    results["totalArtistOptTourDist"]=totalArtistOptTourDist
    
    with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)    
    print totalDists," / ",totalArtistTourDist," / ",totalArtistOptTourDist," / ",totalOptDistsMin," / ",tmppp
print "***************************"
print "TOTAL ARTISTS TOURS DISTS",totalDists
print "TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS",totalOptDists 
print "TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS",totalOptDistsMin 
print "AUTO-OPTIMIZED BY ", 100 *(totalDists-totalOptDists) / totalDists," %"
print "MANUALLY OPTIMIZED BY ", 100 *(totalDists-totalOptDistsMin) / totalDists," %"
print "***************************"


SyntaxError: invalid syntax (<ipython-input-14-83768b6d4756>, line 103)

In [ ]:
#FULL
final_embeddings=[]
totalDists=0
totalOptDists=0
totalOptDistsMin=0
#for artist in db.minedArtists.find({ 'name' : 'David Guetta' }) :
#for artist in db.minedArtists.find({ 'name' : 'Vokab Kompany' }) :
#for artist in db.minedArtists.find({ 'name' : 'Ural Kosaken Chor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Leo Kottke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Björk' }) :
#for artist in db.minedArtists.find({ 'name' : 'Deerhoof' }) :
#for artist in db.minedArtists.find({ 'name' : 'Legends' }) :
#for artist in db.minedArtists.find({ 'name' : 'Laidback Luke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Andy Moor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Mia' }) :
#for artist in db.minedArtists.find({ 'name' : 'Dead Kennedys' }) :
#for artist in db.minedArtists.find({ 'name' : 'House of Pain' }) :
for artist in db.minedArtists.find(no_cursor_timeout=True) :

 if os.path.exists("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        print "SKIPPED",artist["name"]
        print slugify(artist["name"])
        with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          temp= pickle.load( handle)
        #print temp
        tmppp =0
        
        totalArtistTourDist=0
        totalArtistOptTourDist=0
        totalArtistTourDist=temp["totalArtistTourDist"]
        totalArtistOptTourDist=temp["totalArtistOptTourDist"]
        
        totalDists+=totalArtistTourDist
        totalOptDists+=totalArtistOptTourDist
        if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
        else:
            tmppp=float(totalArtistOptTourDist)
        totalOptDistsMin+=tmppp
        #print totalArtistTourDist," / ",totalArtistOptTourDist
        print totalDists," / ",totalOptDists," / ",totalOptDistsMin," / ",tmppp
        
        #pass
 else:
    

    results={}
    tourindice=0
    results["name"]=artist["name"]
    print artist["name"]
    totalArtistTourDist=0
    totalArtistOptTourDist=0
    for tour in artist["tours"]:
        #print tour
        tourCoords= []
        tourdist=0
        tourdistOpt=0
        tourdistOpt2=0
        tourdistOpt3=0
        t=0
        #print len(tour["tourCoords"])
        if len(tour["tourCoords"])>2:
            tourindice+=1
            tourCoords=[tuple(tourCoord) for tourCoord in tour["tourCoords"]]
            #print tourCoords
            ###RERUN OPT,PICKING BETTER SOLUTION
            t=tsp(tourCoords)
            t.solve()
            #print "here"

            #print t.result
            #print t.points
            #print "long",len (t.result)
            i = 0
            while i < len (t.result)-1:
                res=t.result[i]
                res2=t.result[i+1]
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdistOpt= tourdistOpt + vincenty(t.points[res],t.points[res2]).kilometers
                
                
                i+=1
            
            j = 0
            while j < len (tourCoords)-1:
                
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdist= tourdist + vincenty(tourCoords[j],tourCoords[j+1]).kilometers
                j+=1
            
            #print "DEFAULT TOUR DIST",tourdist
            #print "TOURDISTOPTIMIZED",tourdistOpt
            totalArtistTourDist+=tourdist
            totalArtistOptTourDist+=tourdistOpt
            results[tourindice]={'tourdist':tourdist,'tourdistOpt':tourdistOpt}
    #print results
    
    print "TOTAL DEFAULT ARTIST TOURS DISTS",totalArtistTourDist
    print "TOTAL OPTIMIZED ARTIST TOURS DISTS",totalArtistOptTourDist 
    if totalArtistTourDist > 0 :
        OptArtistpercent=0
        
        OptArtistpercent=100 *(totalArtistTourDist-totalArtistOptTourDist) / totalArtistTourDist
        print "OPTIMIZED BY ", ," %"
    print"_______________________________"
    totalDists+=totalArtistTourDist
    totalOptDists+=totalArtistOptTourDist
    tmppp=0
    if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
    else:
            tmppp=float(totalArtistOptTourDist)
    totalOptDistsMin+=tmppp
    
    
    results["totalArtistTourDist"]=totalArtistTourDist
    results["totalArtistOptTourDist"]=totalArtistOptTourDist
    
    with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)    
    print totalDists," / ",totalArtistTourDist," / ",totalArtistOptTourDist," / ",totalOptDistsMin," / ",tmppp
print "***************************"
print "TOTAL ARTISTS TOURS DISTS",totalDists
print "TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS",totalOptDists 
print "TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS",totalOptDistsMin 
print "AUTO-OPTIMIZED BY ", 100 *(totalDists-totalOptDists) / totalDists," %"
print "MANUALLY OPTIMIZED BY ", 100 *(totalDists-totalOptDistsMin) / totalDists," %"
print "***************************"


In [ ]:
with open("./tour_miner/"+"young-mc"+".obj", 'rb') as handle:
          temp= pickle.load( handle)
print temp

In [6]:
with open("./RESULTSGLOBAL.obj", 'rb') as handle:
          temp= pickle.load( handle)
print temp

{'manOpt': 0.23255376595961377, 'autoOpt': 0.21995298276047787, 'totalOptDistsMin': 1639279100.4292355, 'totalOptDists': 1666194602.2991636, 'totalDists': 2136018170.0272305}


In [ ]:
***************************
TOTAL ARTISTS TOURS DISTS 2135250690.17
TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS 1665625398.71
TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS 1638787824.49
AUTO-OPTIMIZED BY  21.993918261  %
MANUALLY OPTIMIZED BY  23.2507999161  %
***************************

***************************
TOTAL ARTISTS TOURS DISTS 2136018170.03
TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS 1666994420.69
TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS 1640164490.57
AUTO-OPTIMIZED BY  21.9578539134  %
MANUALLY OPTIMIZED BY  23.2139260994  %
***************************
